In [1]:
#@title Set up
%%capture
SERVICE_ACCOUNT_FILE = '/content/gcp_api_connector/resources/service_account_credentials.json'

#?
PROJECT_ID = '' #https://console.cloud.google.com
GITHUB_TOKEN = '' #https://github.com/settings/tokens
GITHUB_USER = '' #https://github.com/

!git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/gcp_api_connector.git

!pip install --upgrade pip
!pip install --upgrade google-api-python-client
!pip install pydub
from googleapiclient.discovery import build
from gcp_api_connector.resources.function_importer import generate_code_snippet

In [2]:
#@title Import statements
%%capture
print(generate_code_snippet("gcp_api_connector", False))
from gcp_api_connector.buckets import create_bucket
from gcp_api_connector.buckets import upload_blob
from gcp_api_connector.buckets import list_buckets
from gcp_api_connector.buckets import list_blobs
from gcp_api_connector.buckets import download_bucket
from gcp_api_connector.buckets import download_blob
from gcp_api_connector.buckets import delete_blob
from gcp_api_connector.buckets import empty_bucket
from gcp_api_connector.buckets import delete_bucket

from gcp_api_connector.service_usage import enable_services
from gcp_api_connector.service_usage import list_services
from gcp_api_connector.service_usage import get_service_name

from gcp_api_connector.text_to_speech import help
from gcp_api_connector.text_to_speech import split_text
from gcp_api_connector.text_to_speech import concatenate_audio_files
from gcp_api_connector.text_to_speech import text_to_speech
from gcp_api_connector.text_to_speech import synthesize_speech
from gcp_api_connector.text_to_speech import say

from gcp_api_connector.bigquery import BigQueryTable

from gcp_api_connector.search import search

from gcp_api_connector.translate import en_sp
from gcp_api_connector.translate import sp_en

from gcp_api_connector.vision import read_image

from gcp_api_connector.resources.function_importer import extract_items
from gcp_api_connector.resources.function_importer import generate_code_snippet

from gcp_api_connector.resources.googleapis_auth import get_api_key



In [3]:
#@title service_usage.py
#services_dict = list_services(PROJECT_ID, SERVICE_ACCOUNT_FILE)
#get_service_name("Cloud Vision API", services_dict)

enable_services(PROJECT_ID, SERVICE_ACCOUNT_FILE, ["customsearch", "texttospeech", "translate", "vision"])

Successfully enabled customsearch.googleapis.com for project gcp-api-connector-387814
Successfully enabled texttospeech.googleapis.com for project gcp-api-connector-387814
Successfully enabled translate.googleapis.com for project gcp-api-connector-387814
Successfully enabled vision.googleapis.com for project gcp-api-connector-387814


In [4]:
#@title buckets.py
from google.cloud import storage
storage_client = storage.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)
BUCKET_NAME = "my_example_bucket_f4c89a0ac7000101" # Bucket names must be unique to avoid Forbidden errors.
create_bucket(BUCKET_NAME)
upload_blob(BUCKET_NAME, '/content/gcp_api_connector/resources/Image001.png', 'Image001.png')
list_buckets()
list_blobs(BUCKET_NAME)
download_blob(BUCKET_NAME, 'Image001.png')
download_bucket(BUCKET_NAME, '/content/my_files')
delete_blob(BUCKET_NAME, 'Image001.png')
empty_bucket(BUCKET_NAME)
delete_bucket(BUCKET_NAME)

Bucket my_example_bucket_f4c89a0ac7000101 created
File /content/gcp_api_connector/resources/Image001.png uploaded to Image001.png.
Image001.png
Blob Image001.png downloaded to Image001.png.
Blob Image001.png downloaded to /content/my_files/Image001.png.
Blob Image001.png deleted.
All blobs in my_example_bucket_f4c89a0ac7000101 have been deleted.
Bucket my_example_bucket_f4c89a0ac7000101 deleted


In [5]:
#@title search.py
search('python', 1)

[{'kind': 'customsearch#result',
  'title': 'CS 1110 Fall 2023',
  'htmlTitle': 'CS 1110 Fall 2023',
  'link': 'https://www.cs.cornell.edu/courses/cs1110/',
  'displayLink': 'www.cs.cornell.edu',
  'snippet': 'CS 1110 is the introductory course for computer science and information science majors at Cornell.',
  'htmlSnippet': 'CS 1110 is the introductory course for computer science and information science majors at Cornell.',
  'cacheId': 'tVfetl8iY28J',
  'formattedUrl': 'https://www.cs.cornell.edu/courses/cs1110/',
  'htmlFormattedUrl': 'https://www.cs.cornell.edu/courses/cs1110/',
  'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcRHjGDCffKjk89vLqeVVi6E_psssIQTWNL21QvjpVLkG0EqP0RgXqbSRM2c',
     'width': '300',
     'height': '168'}],
   'metatags': [{'msapplication-tilecolor': '#fabb00',
     'og:site_name': 'CS 1110 Fall 2023',
     'viewport': 'width=device-width, initial-scale=1.0',
     'og:title': 'CS 1110 Fall 2023',
     'og:locale

In [6]:
#@title text_to_speech.py
say("hello world")
say("hola mundo", model='es-ES-Standard-A', filename='output.wav')

In [7]:
#@title translate.py
print(en_sp('hello world'))
print(sp_en('hola mundo'))

Hola Mundo
Hello World


In [8]:
#@title vision.py
print(read_image('/content/gcp_api_connector/resources/Image001.png'))

They say every text has a story.
Mine began on an image.


In [9]:
#@title bigquery.py
table_products = BigQueryTable(
    dataset_id="BaseProducts",
    table_id="TableProducts",
    columns="ID INTEGER REQUIRED, ITEM_NAME STRING REQUIRED, PRICE INTEGER REQUIRED, SECTION STRING REQUIRED",
    SERVICE_ACCOUNT_CREDENTIALS="/content/gcp_api_connector/resources/service_account_credentials.json"
)

table_products.insert_records("NULL, 'BALL', 10, 'SPORT'")

records = [
    "NULL, 'GOLF STICK', 25, 'SPORT'",
    "NULL, 'GLASS', 20, 'CERAMIC'",
    "NULL, 'T-SHIRT', 5, 'CLOTHES'"
]
table_products.insert_records(records)

results = table_products.read_records("PRICE > 20 ORDER BY ID")
print("Before:\n", results)

table_products.execute_command("UPDATE {dataset}.{table} SET PRICE = 21 WHERE ITEM_NAME = 'GLASS'")
table_products.execute_command("DELETE FROM {dataset}.{table} WHERE ID = 2")

results = table_products.read_records("PRICE > 20 ORDER BY ID")
print("After:\n", results)

table_products.delete("table_content")
table_products.delete("table")
table_products.delete("dataset")


Dataset BaseProducts created
Table TableProducts created in dataset BaseProducts
Record inserted into table TableProducts in dataset BaseProducts
Multiple records inserted into table TableProducts in dataset BaseProducts
Before:
 [(2, 'GOLF STICK', 25, 'SPORT')]
After:
 [(3, 'GLASS', 21, 'CERAMIC')]
Table TableProducts contents deleted in dataset BaseProducts
Table TableProducts deleted from dataset BaseProducts
Dataset BaseProducts and its contents deleted successfully
